## Instalando Requisitos

## Imports

In [ ]:
from tabula import read_pdf
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import os

### Monitor de download

In [ ]:
import time

def download_wait(directory, timeout, nfiles=None):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.
    """
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

## Inicializar classe RemuneracoesCampinas com Selenium e Chromium

In [ ]:
class CargosESalariosCampinas:
    def __init__(self):
        self.site_link = "https://cargosesalarios.campinas.sp.gov.br/"
        self.site_map = {
            "buttons":{
                "cargosemcomissao":{
                    "xpath" : "/html/body/div[3]/div/div/div/form/div/ul/li[9]/a"
                }
            }
        }
        self.options = Options()
        self.options.binary_location = f"{os.getcwd()}\\chrome-win\\chrome.exe"
        self.options.add_experimental_option('prefs', {
            "download.default_directory": os.getcwd(), 
            "plugins.always_open_pdf_externally": True #Faz o download ao invez de abrir no chrome
        })

        self.driver = webdriver.Chrome(executable_path=f"{os.getcwd()}\\chromedriver.exe", options = self.options)
        
    def abrir_site(self):
        time.sleep(2)
        self.driver.get(self.site_link)
        time.sleep(7)
        
    def click(self, campo):
        xpath = self.site_map["buttons"][campo]["xpath"]
        self.driver.find_element("xpath", xpath).click()
          

In [ ]:
cs = CargosESalariosCampinas()

In [ ]:
cs.abrir_site()

In [ ]:
cs.click('cargosemcomissao')

In [ ]:
pdf = read_pdf("Tabela Salarial - Cargos em Comissão - Mai22 - ret.pdf", pages="all", encoding="ISO-8859-1")

In [ ]:
df_cargoscomissao = pdf[0]

In [ ]:
df_cargoscomissao['FLAG_COMISSAO'] = True

In [ ]:
df_cargoscomissao.to_csv('cargosemcomissao.csv', index=False)